This notebook collects all the game ID's from 

In [5]:
import json
from urllib.request import urlopen
import pandas as pd
import os
from tqdm.notebook import tqdm

# variables
scrape_details = True # if data should be scraped or loaded from csv
num_apps = 0 # number of reviews to scrape, if 0 all reviews will be scraped
all_games_url = "http://api.steampowered.com/ISteamApps/GetAppList/v0002/?key=STEAMKEY&format=json"
steam_spy_url = "https://steamspy.com/api.php?request=appdetails&appid="


In [6]:
# Scraping the data

if scrape_details:
    # get all app ID
    with urlopen(all_games_url) as url:
        app_ids = pd.json_normalize(json.load(url)["applist"], record_path =['apps'])["appid"]
   
    # check if all should be scraped
    if num_apps == 0: num_apps = len(app_ids)
    df = pd.DataFrame()
    
    # scrap first num_apps reviews
    for appid in tqdm(app_ids.head(num_apps)):
        # create specific detail url
        with urlopen(steam_spy_url + str(appid)) as url:
            # append Details
            df = df.append(pd.DataFrame.from_records([json.load(url)], index="appid"))
    
    df.to_csv('raw_data.csv')

else:
    df = pd.read_csv('raw_data.csv', index_col=0)

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
# Cleaning the data
# games that have no reviews
df.drop(df[df.num_reviews == 0].index, inplace=True)

# games for which information was not fetched
df.drop(df[df.num_reviews == ""].index, inplace=True)
df.drop(df[df.review_score == ""].index, inplace=True)
df.to_csv('clean_data.csv')
